In [ ]:
import ibis
from ibis import _
import streamlit as st
import ibis.expr.datatypes as dt  

conn = ibis.duckdb.connect(extensions=["spatial"])

# Adding county/city parties for 1988 - 1996

In [ ]:
# convert wide format to long format
def convert_data(candidates,year,county):
    county= (county
                .rename(county="Geographic Name", GEOID = "FIPS")
              .filter(_.GEOID != "fips")
             .cast({"GEOID":"string"})
                .mutate(
                    GEOID=ibis.case()
            .when(_.GEOID.length() == 4, "0"+_.GEOID)
                    .else_(_.GEOID)
                    .end()
                )
              .drop("Geographic Subtype")
             .mutate(county= 
                     ibis.case()
                     .when(_.county.endswith('County'), _.county)
                     .else_(_.county + ' County')                     .end())
            )
    
    candidate_labels = ibis.memtable({'candidate': candidates}).cross_join(county)
    
    case_builder = (
        ibis.case()
        .when(candidate_labels.candidate == candidates[0], county[candidates[0]])
    )
    
    for candidate in candidates[1:]:
        case_builder = case_builder.when(candidate_labels.candidate == candidate, county[candidate])
    
    candidate_votes = case_builder.end()
    
    long_format = candidate_labels.mutate(
        GEOID = _.GEOID,
        year=year,
        candidate_votes=candidate_votes.cast("int64")
    ).filter(candidate_votes.notnull() & (candidate_votes.cast("int64") > 0))
    
    result = long_format.select(
        "year", 
        "GEOID",
        "county", 
        "candidate", 
        "candidate_votes"
    )
    return result

In [ ]:
#get winner of each county
def get_winner(df):
    win = (df
        .group_by(["year", "GEOID"])
        .aggregate(candidate=_.candidate.argmax(_.candidate_votes)
                  , county = _.county.max())  # winning party 
    )  
    return win

# 1988 Election Results

In [ ]:
county = conn.read_csv("1988_0_0_2.csv")
         
candidates = ['Michael Dukakis', 'George Bush', 'Ron Paul', 'Lenora Fulani', 'David Duke', 'Eugene McCarthy', 
              'James Griffen', 'Lyndon LaRouche', 'William Marra', 'Write-In', 'Edward Winn', 'James Warren', 
              'Herbert Lewin', 'Earl Dodge', 'Larry Holmes', 'None o.t. Candidates', 'Willa Kenoyer', 
              'Delmar Dennis', 'Jack Herer', 'Louis Youngkeit', 'John Martin']

year = ibis.literal("1988")

year1 = convert_data(candidates,year,county)

winner = get_winner(year1)

df_1988 = (winner
           .mutate(
               party=ibis.case()
                    .when(_.candidate == "George Bush", "REPUBLICAN")  
                    .when(_.candidate == "Michael Dukakis", "DEMOCRAT")  
                    .else_(None)  
                    .end()
            )
          )



# 1992 Election Results

In [ ]:
import ibis
from ibis import _
import ibis.expr.datatypes as dt  

county = (conn
          .read_csv("1992_0_0_2.csv")
         )

candidates = ['William Clinton', 'George Bush', 'H. Ross Perot', 'Andre Marrou', 'James Bo Gritz', 'Lenore Fulani', 
              'Howard Phillips', 'Dr. John Hagelin', 'Ron Daniels', 'Lyndon LaRouche Jr.', 'James Warren', 
              'Write-ins', 'Drew Bradford', 'Jack Herer', 'J. Quinn Brisben', 'Helen Halyard', 
              'None o.t. Candidates', 'John Yiamouyiannis', 'Ehlers', 'Earl Dodge', 'Jim Boren', 
              'Hem', 'Isabell Masters', 'Smith', 'Gloria LaRiva']

year = ibis.literal("1992")

year2 = convert_data(candidates,year,county)

winner = get_winner(year2)

df_1992 = (winner
           .mutate(
               party=ibis.case()
                    .when(_.candidate == "George Bush", "REPUBLICAN")  
                    .when(_.candidate == "William Clinton", "DEMOCRAT")  
                    .when(_.candidate == "H. Ross Perot", "REFORM")  
                    .else_(None)  
                    .end()
            )
          )



# 1996 Election Results

In [ ]:

county = (conn
          .read_csv("1996_0_0_2.csv")
         )

candidates = [
    'William Clinton', 'Robert Dole', 'H. Ross Perot', 'Ralph Nader', 'Harry Browne', 
    'Howard Phillips', 'Dr. John Hagelin', 'Monica Moorehead', 'Marsha Feinland', 
    'Write-ins', 'Charles Collins', 'James Harris', 'None o.t. Candidates', 
    'Dennis Peron', 'Mary Cal Hollis', 'Jerome White', 'Diane Templin', 'Earl Dodge', 
    'A. Peter Crane', 'Just. Ralph Forbes', 'John Birrenback', 'Isabell Masters pHD', 
    'Steve Michael'
]
year = ibis.literal("1996")

year3 = convert_data(candidates,year,county)

winner = get_winner(year3)

df_1996 = (winner
           .mutate(
               party=ibis.case()
                    .when(_.candidate == "Robert Dole", "REPUBLICAN")  
                    .when(_.candidate == "William Clinton", "DEMOCRAT")  
                    .else_(None)  
                    .end()
            )
          )

# Merging with county polygons 

In [ ]:
df_temp = df_1988.union(df_1992)
df_county = df_temp.union(df_1996)

In [ ]:
state_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-state-territory.parquet"
state = (conn
        .read_parquet(state_boundaries)
        .rename(state = "STUSPS", state_name = "NAME")
        .select('state','geometry','state_name',"GEOID")
        .cast({"geometry": "geometry","GEOID":"string"})
        )


# merging with county polygons
county_boundaries = "https://data.source.coop/cboettig/us-boundaries/us-county.parquet"
df_polygons = (conn
            .read_parquet(county_boundaries)
            .rename(county = "NAMELSAD", state_name = "STATE_NAME")
            .join(state, "state_name", how = "inner")
            .select('state','state_name','county','geometry','GEOID')
     )




df_county_polygons = (df_polygons
            .join(df_county, ["GEOID"],how = "inner")
            .mutate(municipal = None)
            .mutate(jurisdiction = ibis.literal("County"))
            .select("state", "county", "municipal","jurisdiction","geometry", "year", "party")
            .cast({"geometry": "geometry","municipal":"string","year":"int64"})
            .mutate(county = _.county.upper())
            )

# Combine with Elections 2000-2020 (Only Counties)

In [ ]:
df_2000 = (conn
             .read_parquet("party_polygons.parquet")
             .cast({"geometry": "geometry"})
             .filter(_.jurisdiction == "County")
             .union(df_county_polygons)
            )


# Make PMTiles - only with county level parties 

In [ ]:
import subprocess
import os
from huggingface_hub import HfApi, login
import streamlit as st

login(st.secrets["HF_TOKEN"])
# api = HfApi(add_to_git_credential=False)
api = HfApi()

def hf_upload(file, repo_id):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
def generate_pmtiles(input_file, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-zg",
        "--extend-zooms-if-still-dropping",
        "--force",
        "--projection", "EPSG:4326",  
        "-L","county:"+input_file
    ]
    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")



In [ ]:
gdf = df_2000.execute().set_crs("EPSG:4326")
gdf.to_file("party_polygons_all.geojson")

generate_pmtiles("party_polygons_all.geojson", "party_polygons_all.pmtiles")
hf_upload("party_polygons_all.pmtiles", "boettiger-lab/landvote")


In [ ]:
# save parquet 
df_2000.execute().set_crs("EPSG:4326").to_parquet("party_polygons_all.parquet")
hf_upload("party_polygons_all.parquet", "boettiger-lab/landvote")


# Getting party affiliation for cities 1988-1996

In [ ]:

         
df_state_city = (df_polygons
         .join(df_county, ["GEOID"],how = "inner")
            .mutate(county = _.county.upper())
                 .select('state','county','year','party')
)


localities_boundaries = "us_localities.parquet"
locality = (conn
            .read_parquet(localities_boundaries)
            .mutate(county = _.county.upper())
            .mutate(municipal = _.municipal.upper())
            .mutate(county=ibis.case()
                 .when(_.county.endswith('COUNTY'), _.county)
                 .else_(_.county + ' COUNTY')
                .end())
           )

df_city = (df_state_city
            # .drop("state_name")
            .join(locality, ["state","county"], how = "inner")
            .mutate(jurisdiction = ibis.literal("Municipal"))
            .select("state", "county", "municipal","jurisdiction","geometry", "year", "party")
            .cast({"geometry": "geometry","municipal":"string","year":"int64"})
            )


df_before2000 = df_city.union(df_county_polygons) # adding all the data before 2000


# Combine with Elections 2000-2020 (Cities + Counties)

In [ ]:

df_2000_all = (conn
           .read_parquet("party_polygons.parquet")
           .cast({"geometry": "geometry"})
            .union(df_before2000)
            )


In [ ]:
# save parquet 
df_2000_all.execute().set_crs("EPSG:4326").to_parquet("party_polygons_all.parquet")
hf_upload("party_polygons_all.parquet", "boettiger-lab/landvote")
